In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup, SoupStrainer
import sys
import string
import requests
import datetime
from datetime import timedelta
import time

In [2]:
## backfill
first_game = datetime.datetime.strptime('2019-10-22', "%Y-%m-%d")
last_game = datetime.datetime.strptime('2020-04-15', "%Y-%m-%d")
winning_team = 'Toronto'
pd.options.display.max_colwidth = 100
game_date = first_game

columns = ['date','Winner', 'Winner_Score', 'Loser', 'Loser_Score', 'box_score']
df = pd.DataFrame(columns = columns)

while game_date < datetime.datetime.now() and datetime.datetime.now() <= last_game:
    d = game_date.day
    m = game_date.month
    y = game_date.year
    date_str = str(y) + '-' + str(m) + '-' + str(d)
    
    base_url = 'http://basketball-reference.com/boxscores/index.fcgi?month=' + str(m) + '&day=' +str(d)+'&year=' + str(y)
    page_response = requests.get(base_url, timeout=5)
    page_content = BeautifulSoup(page_response.content, "html.parser")
    
    scores = page_content.find_all('table', {'class': 'teams'})
    
    for games in scores:
        if winning_team in str(games):
            data = []
            champ_score = games

            winner = champ_score.find('tr', {'class':'winner'}) ## find row that has the winner
            winning_team = winner.find('a').string ## winning team name
            winning_score = winner.find('td', {'class':'right'}).string ## winning team score

            loser = champ_score.find('tr', {'class':'loser'}) ## find row that has the loser
            losing_team = loser.find('a').string ## losing team name
            losing_score = loser.find('td', {'class':'right'}).string ## losing team score

            box_score_td = champ_score.find('td', {'class':['gamelink']}) ## find the box score link
            href = box_score_td.find('a')['href'] ## get the href from the anchor tag
            box_score_url = 'https://www.basketball-reference.com' + href
            # anchor_tag = href='https://www.basketball-reference.com" + href + "'>" + 'Link</a>'


            data.insert(0, {'date': date_str,
                            'Winner': winning_team,
                            'Winner_Score': winning_score,
                            'Loser': losing_team,
                            'Loser_Score': losing_score,
                            'box_score': box_score_url})
            df = pd.concat([pd.DataFrame(data), df], ignore_index=True)
            break
        else:
            continue

    game_date = game_date + timedelta(days=1)

columns = ['date','Winner', 'Winner_Score', 'Loser', 'Loser_Score', 'box_score']
df = df[columns] # reorder columns
# df = df.set_index('date') # set index to date

In [8]:
df.head()

,date,Winner,Winner_Score,Loser,Loser_Score,box_score
0,2020-3-10,Boston,114,Indiana,111,https://www.basketball-reference.com/boxscores/202003100IND.html
1,2020-3-8,Indiana,112,Dallas,109,https://www.basketball-reference.com/boxscores/202003080DAL.html
2,2020-3-6,Dallas,121,Memphis,96,https://www.basketball-reference.com/boxscores/202003060DAL.html
3,2020-3-4,Memphis,118,Brooklyn,79,https://www.basketball-reference.com/boxscores/202003040BRK.html
4,2020-3-2,Memphis,127,Atlanta,88,https://www.basketball-reference.com/boxscores/202003020ATL.html


In [4]:
## find winner
belt_holder = df['Winner'][0]
print("Current championship belt holder: " + belt_holder)

Current championship belt holder: Boston


In [5]:
## find team with the most title defenses
winner_count = df.groupby('Winner').count()[['Winner_Score']] ## get count of each games won while holding belt
winner_count = winner_count.sort_values('Winner_Score', axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last') ## reorder
winner_count = winner_count.reset_index(drop=False) ## remove index so Winner header can be called
most_win_team = winner_count['Winner'][0]
most_wins = winner_count['Winner_Score'][0]

print("The team with the most title defenses: " + most_win_team)
print("They have " + str(most_wins) + " title defense wins.")

The team with the most title defenses: Boston
They have 15 title defense wins.


In [6]:
winner_count.head()

,Winner,Winner_Score
0,Boston,15
1,Memphis,10
2,Indiana,6
3,LA Lakers,5
4,Milwaukee,5


In [26]:
new_df = df
new_df.head()

,date,Winner,Winner_Score,Loser,Loser_Score,box_score
0,2020-3-10,Boston,114,Indiana,111,https://www.basketball-reference.com/boxscores/202003100IND.html
1,2020-3-8,Indiana,112,Dallas,109,https://www.basketball-reference.com/boxscores/202003080DAL.html
2,2020-3-6,Dallas,121,Memphis,96,https://www.basketball-reference.com/boxscores/202003060DAL.html
3,2020-3-4,Memphis,118,Brooklyn,79,https://www.basketball-reference.com/boxscores/202003040BRK.html
4,2020-3-2,Memphis,127,Atlanta,88,https://www.basketball-reference.com/boxscores/202003020ATL.html


In [10]:
new_df = new_df.set_index('date') # set index to date

In [13]:
new_df = new_df.reset_index(drop=False)

In [22]:
new_df = new_df.reset_index(drop=True, inplace=True)

In [23]:
new_df

In [25]:
new_df.size()

AttributeError: 'NoneType' object has no attribute 'size'

In [27]:
df2 = new_df.reset_index()

In [28]:
df2

,index,date,Winner,Winner_Score,Loser,Loser_Score,box_score
0,0,2020-3-10,Boston,114,Indiana,111,https://www.basketball-reference.com/boxscores/202003100IND.html
1,1,2020-3-8,Indiana,112,Dallas,109,https://www.basketball-reference.com/boxscores/202003080DAL.html
2,2,2020-3-6,Dallas,121,Memphis,96,https://www.basketball-reference.com/boxscores/202003060DAL.html
3,3,2020-3-4,Memphis,118,Brooklyn,79,https://www.basketball-reference.com/boxscores/202003040BRK.html
4,4,2020-3-2,Memphis,127,Atlanta,88,https://www.basketball-reference.com/boxscores/202003020ATL.html
5,5,2020-2-29,Memphis,105,LA Lakers,88,https://www.basketball-reference.com/boxscores/202002290MEM.html
6,6,2020-2-27,LA Lakers,116,Golden State,86,https://www.basketball-reference.com/boxscores/202002270GSW.html
7,7,2020-2-25,LA Lakers,118,New Orleans,109,https://www.basketball-reference.com/boxscores/202002250LAL.html
8,8,2020-2-23,LA Lakers,114,Boston,112,https://www.basketball-reference.com/boxscores/202002230LAL.html
9,9,2020-2-21,LA Lakers,117,Memphis,105,https://www.basketball-reference.com/boxscores/202002210LAL.html
